In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from sklearn.model_selection import train_test_split
from tqdm import tqdm
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

from transformers import T5ForConditionalGeneration,T5Tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


device(type='cpu')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Load dataset
dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Dataset/news_summary.csv", encoding='latin1')

In [6]:
train_dataset, test_dataset = train_test_split(dataset, shuffle=True, test_size=0.2, random_state=42)
train_dataset, val_dataset = train_test_split(train_dataset, shuffle=True, test_size=0.1, random_state=42)

print(f"Train set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

Train set size: 70848
Validation set size: 7872
Test set size: 19681


In [7]:
model = T5ForConditionalGeneration.from_pretrained("Michau/t5-base-en-generate-headline")
tokenizer = T5Tokenizer.from_pretrained("Michau/t5-base-en-generate-headline")
model = model.to(device)

config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
# Function to generate headlines
def generate_headline(text):
    encoding = tokenizer.encode_plus("headline: " + text, return_tensors = "pt")
    input_ids = encoding["input_ids"].to(device)
    attention_masks = encoding["attention_mask"].to(device)
    outputs = model.generate(input_ids = input_ids, attention_mask = attention_masks,
                             max_length=100, min_length=20)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [9]:
def evaluateRandomly_test(n=10):
    for i in range(n):
        print(i)
        eval_sample = test_dataset.iloc[i:i+1, :]
        print('news_article > ', eval_sample['text'].iloc[0])
        headline = eval_sample['headlines'].iloc[0]
        print('original_headline = ', headline)
        output_sentence = generate_headline(eval_sample['text'].iloc[0])
        print('predicted_headline < ', output_sentence)
        print(f"meteor score: {nltk.translate.meteor_score.single_meteor_score(headline.split(), output_sentence.split())}")

In [10]:
evaluateRandomly_test(n=10)

0
news_article >  Students in Karnataka will get extra marks if their parents cast votes in the upcoming assembly elections, the Associated Management of Primary and Secondary Schools has announced. The "Encouraging Marks" will be added in the 2018-19 academic year. The association said, "After casting their votes, parents can visit member schools...and confirm that they voted by showing the indelible ink mark."
original_headline =  K'taka students to get extra marks if parents vote in polls
predicted_headline <  Karnataka - Parents Can Vote in Assembly Elections to Get Extra Marks
meteor score: 0.6058098915241772
1
news_article >  Syrian anti-aircraft defences on Monday shot down missiles over two air bases, Syria's state media said. The missiles targeted Shayrat air base in the Homs province and another base northeast of the capital Damascus. This comes days after the US, UK and France launched air strikes on Syrian chemical weapons facilities in retaliation for the alleged chemical 